In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
tf.keras.backend.clear_session()
print(tf.__version__)

2.0.0


In [2]:
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
inputs = keras.Input(shape=(784,),name = 'digits')
x = layers.Dense(64,activation='relu',name='dense_1')(inputs)
x = layers.Dense(64,activation='relu',name = 'dense_2')(x)
outputs = layers.Dense(10,activation='softmax',name = 'predictions')(x)
model = keras.Model(inputs = inputs,outputs = outputs)

In [4]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

In [5]:
model.compile(optimizer=keras.optimizers.RMSprop(),
             loss = keras.losses.SparseCategoricalCrossentropy(),
             metrics = [keras.metrics.SparseCategoricalAccuracy()])

In [6]:
history = model.fit(x_train,y_train,
                   batch_size=64,
                   epochs=3,
                   validation_data=(x_val,y_val))
print('\n histroy dict: ',history.history)

Train on 50000 samples, validate on 10000 samples
Epoch 1/3
50000/50000 [==============================] - 5s 109us/sample - loss: 0.3410 - sparse_categorical_accuracy: 0.9021 - val_loss: 0.2136 - val_sparse_categorical_accuracy: 0.9317
Epoch 2/3
50000/50000 [==============================] - 4s 78us/sample - loss: 0.1618 - sparse_categorical_accuracy: 0.9516 - val_loss: 0.1531 - val_sparse_categorical_accuracy: 0.9552
Epoch 3/3
50000/50000 [==============================] - 4s 78us/sample - loss: 0.1188 - sparse_categorical_accuracy: 0.9647 - val_loss: 0.1121 - val_sparse_categorical_accuracy: 0.9675

 histroy dict:  {'loss': [0.3410291019141674, 0.16183276713609696, 0.11883255400687456], 'sparse_categorical_accuracy': [0.90206, 0.95164, 0.96472], 'val_loss': [0.21358426371514797, 0.1530543849349022, 0.11210459920093417], 'val_sparse_categorical_accuracy': [0.9317, 0.9552, 0.9675]}


In [7]:
print('\n Evaluate on test data')
results = model.evaluate(x_test,y_test,batch_size=128)
print('test loss,test auc::',results)


 Evaluate on test data
10000/1 [=======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

test loss,test auc:: [0.10842638254500926, 0.9663]


In [8]:
print('\n Generate predictions for 3 samples')
predictions = model.predict(x_test[:3])
print('predictions shape:',predictions.shape)


 Generate predictions for 3 samples
predictions shape: (3, 10)


In [9]:
predictions

array([[3.6889406e-07, 6.8370187e-10, 1.2990298e-04, 1.6806800e-04,
        3.6202510e-09, 7.7259557e-07, 5.8535465e-14, 9.9966741e-01,
        5.7944254e-07, 3.3001481e-05],
       [7.6821046e-08, 9.5462252e-05, 9.9985611e-01, 1.1218480e-05,
        1.4770174e-12, 7.5532483e-07, 2.3540802e-07, 7.2615364e-10,
        3.6154514e-05, 5.2385169e-13],
       [4.0685049e-06, 9.9670416e-01, 6.0986960e-04, 8.8880981e-05,
        9.5845782e-05, 4.5450288e-05, 4.2089432e-05, 1.6178272e-03,
        7.6761423e-04, 2.4086952e-05]], dtype=float32)

In [10]:
def get_uncompiled_model():
    inputs = keras.Input(shape=(784,),name = 'digits')
    x = layers.Dense(64,activation='relu',name='dense_1')(inputs)
    x = layers.Dense(64,activation='relu',name = 'dense_2')(x)
    outputs = layers.Dense(10,activation='softmax',name = 'predictions')(x)
    model = keras.Model(inputs = inputs,outputs = outputs)
    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
    return model

In [11]:
def basic_loss_function(y_true,y_pred):
    return tf.math.reduce_mean(y_true - y_pred)

model.compile(optimizer=keras.optimizers.Adam(),
             loss = basic_loss_function)
model.fit(x_train,y_train,batch_size = 64,epochs=3)

Train on 50000 samples
Epoch 1/3
50000/50000 [==============================] - 4s 71us/sample - loss: 4.3488
Epoch 2/3
50000/50000 [==============================] - 3s 53us/sample - loss: 4.3488
Epoch 3/3
50000/50000 [==============================] - 3s 52us/sample - loss: 4.3488


In [13]:
class WeightedBinaryCrossEntropy(keras.losses.Loss):
    """
    Args:
      pos_weight: Scalar to affect the positive labels of the loss function.
      weight: Scalar to affect the entirety of the loss function.
      from_logits: Whether to compute loss form logits or the probability.
      reduction: Type of tf.keras.losses.Reduction to apply to loss.
      name: Name of the loss function.
    """
    def __init__(self,pos_weight,weight,from_logits = False,
                reduction = keras.losses.Reduction.AUTO,
                name = 'weighted_binary_crossentropy'):
        super(WeightedBinaryCrossEntropy,self).__init__(
        reduction=reduction,name=name)
        self.pos_weight = pos_weight
        self.weight = weight
        self.from_logits = from_logits
        
    def call(self,y_true,y_pred):
        if not self.from_logits:
           # Manually calculate the weighted cross entropy.
            # Formula is qz * -log(sigmoid(x)) + (1 - z) * -log(1 - sigmoid(x))
            # where z are labels, x is logits, and q is the weight.
            # Since the values passed are from sigmoid (assuming in this case)
            # sigmoid(x) will be replaced by y_pred

            # qz * -log(sigmoid(x)) 1e-6 is added as an epsilon to stop passing a zero into the log
            x_1 = y_true*self.pos_weight*tf.math.log(y_pred+1e-6)
            x_2 = (1-y_true)*-tf.math.log(1-y_pred+1e-6)
            return tf.add(x_1,x_2)*self.weight
        return tf.nn.weighted_cross_entropy_with_logits(y_true,y_pred,sel.pos_weight)*self.weight
    
        

In [14]:
model.compile(optimizer=keras.optimizers.Adam(),
             loss = WeightedBinaryCrossEntropy(0.5,2))
model.fit(x_train,y_train,batch_size=64,epochs=3)

Train on 50000 samples
Epoch 1/3
50000/50000 [==============================] - 4s 79us/sample - loss: -63.0772
Epoch 2/3
50000/50000 [==============================] - 4s 75us/sample - loss: -64.1398
Epoch 3/3
50000/50000 [==============================] - 4s 72us/sample - loss: -64.2941
